In [1]:
# %matplotlib inline

In [2]:
 %matplotlib --list

Available matplotlib backends: ['osx', 'qt4', 'qt5', 'gtk3', 'notebook', 'wx', 'qt', 'nbagg', 'gtk', 'tk', 'inline']


In [22]:
# analyze the focus values for a montage
import cv2
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import json
import glob
import os
from subprocess import call

min_v_to_plot = 37
#cmap = plt.cm.hot 
cmap = plt.cm.rainbow
scale = 8
trows = 0
tcols = 0
scaled_montage_w = 0
scaled_montage_h = 0
root_dir = ""

def display_surface(img):
    fig=plt.figure()
    ax = Axes3D(fig)
    w,h = img.shape
    xx, yy = np.mgrid[0:w, 0:h]
    
    # plot3D requires a 1D array for x, y, and z
    # ravel() converts the 100x100 array into a 1x10000 array
    #ax.plot_surface(xx,yy,img,  cmap=plt.cm.jet)
    ax.plot_surface(xx,yy,img, rstride=2, cstride=2, linewidth=0, cmap=cmap, vmin=min_v_to_plot, antialiased=False,  shade=True, )
    #ax.plot_wireframe(xx,yy,img, linewidth=1, cmap=cmap)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    fig.add_axes(ax)
    plt.show()

def get_meta_and_montage_files(rootdir):
    '''get the names of the meta and montage files'''
    for name in glob.glob(os.path.join(rootdir, r"_meta*.*")):
        meta = name
    for name in glob.glob(os.path.join(rootdir, r"_montage*.*")):
        montage = name
    return (meta, montage)


def process_one(rootdir):
    global trows, tcols, scaled_montage_h, scaled_montage_w

    meta, montage = get_meta_and_montage_files(rootdir)

    with open(meta) as data_file:
        json_data = json.load(data_file)

    data = json_data[1]['data']

    # figure out the number of rows and columns
    trows = tcols = 0
    for tile in data:
        rp = tile['img_meta']['raster_pos']
        col = rp[0]
        row = rp[1]
        #print row, col
        if row > trows:
            trows = row
        if col > tcols:
            tcols = col

    print trows, tcols

    img = np.empty((int(trows + 1), int(tcols+1)))
    img.fill(min_v_to_plot)

    for tile in data:
        rp = tile['img_meta']['raster_pos']
        col = rp[0]
        row = rp[1]
        focus = tile["focus_score"]
        if focus < min_v_to_plot:  # danger, removing data!!!
            focus = min_v_to_plot
        #print focus
        img[row][col] = focus

    # MONTAGE
    imgmontage = cv2.imread(montage, flags=cv2.IMREAD_GRAYSCALE)
    h, w = imgmontage.shape

    imgmontage = cv2.resize(imgmontage, (w/scale, h/scale) , interpolation = cv2.INTER_AREA)
    scaled_montage_h, scaled_montage_w = imgmontage.shape
    imgmontage = cv2.equalizeHist(imgmontage)
    imgmontage = cv2.cvtColor(imgmontage, cv2.COLOR_GRAY2BGR)

    # Focus values
    img = cv2.resize(img, (scaled_montage_w, scaled_montage_h) , interpolation = cv2.INTER_AREA)
    imin = np.min(img)
    imax = np.max(img)
    idelta = imax - imin

    img2 = np.array((img - imin) / idelta * 255.0, dtype='uint8')

    img = img.astype('uint8')
    print img.shape
    img = cv2.applyColorMap(img2, cv2.COLORMAP_JET)
    print img.shape

    print imgmontage.shape
    print img.shape

    fraction = 0.7
    dst = cv2.addWeighted(imgmontage, fraction, img, 1-fraction, 0)
    cv2.imshow(rootdir, dst)
    cv2.waitKey(-1)

def applyCustomColorMap(im_gray) :
    ''' pseudocolor'''
    lut = np.zeros((256, 1, 3), dtype=np.uint8)
    lut[:, 0, 0] = [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,253,251,249,247,245,242,241,238,237,235,233,231,229,227,225,223,221,219,217,215,213,211,209,207,205,203,201,199,197,195,193,191,189,187,185,183,181,179,177,175,173,171,169,167,165,163,161,159,157,155,153,151,149,147,145,143,141,138,136,134,132,131,129,126,125,122,121,118,116,115,113,111,109,107,105,102,100,98,97,94,93,91,89,87,84,83,81,79,77,75,73,70,68,66,64,63,61,59,57,54,52,51,49,47,44,42,40,39,37,34,33,31,29,27,25,22,20,18,17,14,13,11,9,6,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    lut[:, 0, 1] = [ 255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,254,252,250,248,246,244,242,240,238,236,234,232,230,228,226,224,222,220,218,216,214,212,210,208,206,204,202,200,198,196,194,192,190,188,186,184,182,180,178,176,174,171,169,167,165,163,161,159,157,155,153,151,149,147,145,143,141,139,137,135,133,131,129,127,125,123,121,119,117,115,113,111,109,107,105,103,101,99,97,95,93,91,89,87,85,83,82,80,78,76,74,72,70,68,66,64,62,60,58,56,54,52,50,48,46,44,42,40,38,36,34,32,30,28,26,24,22,20,18,16,14,12,10,8,6,4,2,0 ]
    lut[:, 0, 2] = [195,194,193,191,190,189,188,187,186,185,184,183,182,181,179,178,177,176,175,174,173,172,171,170,169,167,166,165,164,163,162,161,160,159,158,157,155,154,153,152,151,150,149,148,147,146,145,143,142,141,140,139,138,137,136,135,134,133,131,130,129,128,127,126,125,125,125,125,125,125,125,125,125,125,125,125,125,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126,126]
    print lut.shape
    im_color = cv2.LUT(im_gray, lut)
    return im_color;

def heatmap_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        click_row = int(y / float(scaled_montage_h) * (trows+1))
        click_col = int(x / float(scaled_montage_w) * (tcols+1))
        print "row: {}, col: {}".format(click_row, click_col)
        for name in glob.glob(os.path.join(rootdir, r"*_{}_{}.tif").format(click_col, click_row)):
            print name
        if name:
            call([r"C:\Program Files (x86)\IrfanView\i_view32.exe", name])


Z = np.arange(256, dtype='uint8').reshape(16,16)
Z = cv2.merge((Z,Z,Z))
#print Z.shape
#print Z

P =  applyCustomColorMap(Z)
cv2.imshow('pseudo', P)
cv2.waitKey(-1)

            
start = 25 #25
end = 25
for i in range (start, end+1):
    rootdir = r"X:\1x1mm_data\239849_7R_0031_01\0000" + str(i) + r"\0"
    cv2.namedWindow(rootdir)
    cv2.setMouseCallback(rootdir, heatmap_click)
    
    process_one(rootdir)

(256L, 1L, 3L)


UnboundLocalError: local variable 'meta' referenced before assignment